In [ ]:
! pip install boto3

# enter values and run this Cell

In [ ]:
import pandas as pd
import os
from datetime import datetime
import numpy as np
import boto3
import shutil
import glob
import datetime as dt
from datetime import date, datetime, timedelta


bucket_name = 'sensus3-91a4f63b-9cc4-4213-b64c-8cbe72a52dc7'

enter_aws_access_key_id ='*******************'

enter_aws_secret_access_key ='********************'

#NO SPACE between group & number
group_number = 'group8'

# path to the folder to save NO SAPCE in folder name
save_folder_path = '/Users/santhoshinigoskula/Documents'

# the folder names in the data folder your group members names as apear on AWS S3
#for example
participent_list = ['Proj_0688','Proj_0905','proj_0914','proj_0922','proj_1068','proj_4830','proj_6815']

# Downloading the data

In [ ]:
s3 = boto3.resource('s3',
                 aws_access_key_id= enter_aws_access_key_id,
                 aws_secret_access_key= enter_aws_secret_access_key)


#     ## Bucket to use
bucket = s3.Bucket(bucket_name)


#get participant list
def get_participant_list(bucket_input):
    client = boto3.client('s3',
                     aws_access_key_id= enter_aws_access_key_id,
                     aws_secret_access_key= enter_aws_secret_access_key)
    result = client.list_objects(Bucket=bucket_input, Prefix='data/', Delimiter='/')
    part_list = []
    for o in result.get('CommonPrefixes'):
        part_list.append(o.get('Prefix').split('/')[1])
    return part_list

#update this to teh correct format


#path of the files
file_list=[]


# def get_objects(part_name):
#     for obj in bucket.objects.filter(Delimiter='/', Prefix=f'data/{part_name}/'):
#         file_list.append(obj.key)

# for i in range(len(participent_list)):
#     get_objects(participent_list[i])
def download_dir(prefix, local, bucket):
    client = boto3.client(
        "s3",
        region_name='us-east-1',
        aws_access_key_id=enter_aws_access_key_id,
        aws_secret_access_key=enter_aws_secret_access_key
    )

    print(f"\n🔎 Checking prefix: {prefix}")
    keys = []
    dirs = []
    next_token = ""
    base_kwargs = {
        "Bucket": bucket,
        "Prefix": prefix,
    }

    while next_token is not None:
        kwargs = base_kwargs.copy()
        if next_token != "":
            kwargs.update({"ContinuationToken": next_token})

        results = client.list_objects_v2(**kwargs)
        contents = results.get("Contents")

        # ✅ If S3 found nothing under this prefix, stop gracefully
        if not contents:
            print(f"⚠️  No objects found for prefix: {prefix}")
            return

        for item in contents:
            k = item.get("Key")
            if k.endswith("/"):
                dirs.append(k)
            else:
                keys.append(k)

        next_token = results.get("NextContinuationToken")

    # Create local directories
    for d in dirs:
        dest_pathname = os.path.join(local, d)
        os.makedirs(os.path.dirname(dest_pathname), exist_ok=True)

    # Download files
    for k in keys:
        dest_pathname = os.path.join(local, k)
        os.makedirs(os.path.dirname(dest_pathname), exist_ok=True)
        client.download_file(bucket, k, dest_pathname)
        print(f"⬇️  Downloaded: {k}")


date_folder = "20251122"   # from your S3 keys

participent_list = [
    "Proj_0688",
    "Proj_0905",
    "Proj_0922",
    "proj_0914",
    "proj_1068",
    "proj_4830",
    "proj_6815",
]

print("Participants to download:", participent_list)

for pid in participent_list:
    prefix = f"data/{date_folder}/{pid}"
    print("⬇️  Starting download for prefix:", prefix)
    download_dir(prefix, save_folder_path, bucket_name)

Participants to download: ['Proj_0688', 'Proj_0905', 'Proj_0922', 'proj_0914', 'proj_1068', 'proj_4830', 'proj_6815']
⬇️  Starting download for prefix: data/20251122/Proj_0688

🔎 Checking prefix: data/20251122/Proj_0688
⬇️  Downloaded: data/20251122/Proj_0688/04487ac7-c29f-48c9-89da-d2c660f98735.json.gz
⬇️  Downloaded: data/20251122/Proj_0688/182aa276-839f-4ffc-b93c-5c1d97341f79.json.gz
⬇️  Downloaded: data/20251122/Proj_0688/f0561c70-07cb-4041-8a60-468059962bdf.json.gz
⬇️  Starting download for prefix: data/20251122/Proj_0905

🔎 Checking prefix: data/20251122/Proj_0905
⬇️  Downloaded: data/20251122/Proj_0905/808a1e25-1c37-40b4-805f-4c8da3e389b6.json.gz
⬇️  Starting download for prefix: data/20251122/Proj_0922

🔎 Checking prefix: data/20251122/Proj_0922
⬇️  Downloaded: data/20251122/Proj_0922/6c34ff7d-ccd5-4ebb-953e-8c349e91ab42.json.gz
⬇️  Starting download for prefix: data/20251122/proj_0914

🔎 Checking prefix: data/20251122/proj_0914
⬇️  Downloaded: data/20251122/proj_0914/6b43406f-

# Creating the data set and output CSV

In [ ]:
#Creating the dataset function
def create_combined_dataset(path_to_raw_data_folder):
    data = [pd.read_json(f'{path_to_raw_data_folder}/{i}') for i in os.listdir(path_to_raw_data_folder) if i.endswith('.gz')]
    return pd.concat(data)

# Function to fix timestamp from UTC to EST
def fix_time(a):
    # If timestamp is missing or empty, return NaT
    if pd.isna(a) or a == "":
        return pd.NaT

    # Try to parse and convert; if it fails, return NaT
    try:
        b = pd.to_datetime(a, utc=True)
        if pd.isna(b):
            return pd.NaT

        # Convert UTC → America/New_York
        c = b.tz_convert("America/New_York")
        # Return formatted string (or keep as datetime if you prefer)
        return c.strftime("%Y-%m-%d %H:%M:%S")
    except Exception:
        return pd.NaT


try:
    os.mkdir(save_folder_path+'/data/allfiles')
except:
    pass
#Move all the results into a single folder
for j in range(len(participent_list)):
    date_folder = "20251122"  # same one you used earlier

for j in range(len(participent_list)):
    pid = participent_list[j]

    source_dir = os.path.join(save_folder_path, "data", date_folder, pid)
    target_dir = os.path.join(save_folder_path, "data", "allfiles")

    # Make sure target_dir exists
    os.makedirs(target_dir, exist_ok=True)

    if not os.path.exists(source_dir):
        print(f"⚠️ Source folder does not exist, skipping: {source_dir}")
        continue

    file_names = os.listdir(source_dir)

    for file_name in file_names:
        shutil.copy(os.path.join(source_dir, file_name), target_dir)

    print(f"✅ Copied files from {source_dir} to {target_dir}")

# find corrupt files, move them to a corrupt folder
zip_file_names = list(glob.glob(target_dir+"/*.gz"))
corrupt_files = []
ok_files_count=0
corrupt_file_count = 0
for i in range(len(zip_file_names)):
    try:
        pd.read_json(zip_file_names[i])
        ok_files_count += 1
        #print(zip_file_names[i] +' ---> FILE OK')
    except:
        corrupt_files.append(zip_file_names[i])
        #print(zip_file_names[i]+' ---> CORRUPTED')
        corrupt_file_count += 1

################################ folders for files
try:
    os.mkdir(save_folder_path+'/data/corrupt_file_folder')
except:
    pass
try:
    for i in range(len(corrupt_files)):
        shutil.move(f"{corrupt_files[i]}", f"{save_folder_path}/data/corrupt_file_folder")
except:
    pass


# In[ ]:


df = create_combined_dataset(target_dir)

df.rename(columns={'$type':'Type'},inplace=True) #fixing the name

df.rename(columns={'Level':'BatteryLevel'},inplace=True) #fixing the level name

df.rename(columns={'Accuracy':'LocationAccuracy'},inplace=True) #fixing the accuracy name

# this is a metadata column that can remove
try:
    df.drop('ProbeParticipation', axis=1, inplace=True)
except:
    pass


# If the column protocol name has the protocol name we remove it since every
# participent is in the same study
try:
    df= df[df['ProtocolName'].isnull()] #removing info row
except:
    pass

#
df.index = pd.RangeIndex(len(df.index)) #fixing the index
df.index.name = 'Row_id' #adding a new id row
df['Formatted_time'] = df['Timestamp'].apply(lambda x: fix_time(x)) # adding a new column changing UTC to EST
df['Formatted_time'] = pd.to_datetime(df['Formatted_time'] ) # fix data type


# remove extra
df['Type'] = df['Type'].replace(regex = {'Sensus.Participation':''})
df['Type'] = df['Type'].replace(regex = {' SensusiOS':''})
df['Type'] = df['Type'].replace(regex = {'Sensus.Probes.Device.': ''})
df['Type'] = df['Type'].replace(regex = {'Sensus.Probes.Location.': ''})
df['Type'] = df['Type'].replace(regex = {'Sensus.Probes.Movement.':''})
df['Type'] = df['Type'].replace(regex = {'Sensus.Probes.Network.': ''})
df['Type'] = df['Type'].replace(regex = {'Sensus.Probes.Context.': ''})
df['Type'] = df['Type'].replace(regex = {'Datum,':''})

# removing metadata rows
df = df[df['Type'] != 'Sensus.Heartbeat']
df = df[df['Type'] != 'Report']

### fixing the values for mysql
df['Timestamp'] = pd.to_datetime(df['Timestamp'],utc=True)
df['Formatted_time'] = pd.to_datetime(df['Formatted_time'] )

    # fixing the order
try:
    df = df[['Type','Id','DeviceId','Timestamp','ProtocolId','BuildId','ParticipantId','DeviceManufacturer',
         'DeviceModel','OperatingSystem','TaggedEventId','TaggedEventTags','SensingAgentStateDescription',
         'LocalOffsetFromUTC','Decibels','BatteryLevel','AccessPointBSSID','Activity','Phase','State',
         'Confidence','Latitude','Longitude','LocationAccuracy','ProtocolName','Formatted_time']]
except:
    df = df[['Type','Id','DeviceId','Timestamp','ProtocolId','BuildId','ParticipantId','DeviceManufacturer',
         'DeviceModel','OperatingSystem','TaggedEventId','TaggedEventTags','SensingAgentStateDescription',
         'LocalOffsetFromUTC','Decibels','BatteryLevel','AccessPointBSSID','Activity','Phase','State',
         'Confidence','ProtocolName','Formatted_time']]

df.to_csv(f'{save_folder_path}/{group_number}_output.csv')

shutil.rmtree(f'{save_folder_path}/data/allfiles')


✅ Copied files from /Users/santhoshinigoskula/Documents/data/20251122/Proj_0688 to /Users/santhoshinigoskula/Documents/data/allfiles
✅ Copied files from /Users/santhoshinigoskula/Documents/data/20251122/Proj_0905 to /Users/santhoshinigoskula/Documents/data/allfiles
✅ Copied files from /Users/santhoshinigoskula/Documents/data/20251122/Proj_0922 to /Users/santhoshinigoskula/Documents/data/allfiles
✅ Copied files from /Users/santhoshinigoskula/Documents/data/20251122/proj_0914 to /Users/santhoshinigoskula/Documents/data/allfiles
✅ Copied files from /Users/santhoshinigoskula/Documents/data/20251122/proj_1068 to /Users/santhoshinigoskula/Documents/data/allfiles
✅ Copied files from /Users/santhoshinigoskula/Documents/data/20251122/proj_4830 to /Users/santhoshinigoskula/Documents/data/allfiles
✅ Copied files from /Users/santhoshinigoskula/Documents/data/20251122/proj_6815 to /Users/santhoshinigoskula/Documents/data/allfiles
